In [1]:
%matplotlib nbagg

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets
from jupyter_ui_poll import ui_events

from libertem.api import Context

from libertem_live.api import LiveContext
from libertem_live.detectors.dectris.acquisition import DectrisAcquisition
from libertem.udf.sumsigudf import SumSigUDF
from libertem.udf.sum import SumUDF
from libertem.udf.masks import ApplyMasksUDF
from libertem.udf.base import UDF
from libertem.viz.bqp import BQLive2DPlot
from libertem.viz.mpl import MPLLive2DPlot
from libertem.analysis import com as com_analysis
from libertem.masks import circular

from libertem_live.udf.monitor import SignalMonitorUDF

from microscope_calibration.udf.stem_overfocus import OverfocusUDF, OverfocusParams, OverfocusProcessParams

In [3]:
ctx = Context()

In [4]:
ds = ctx.load('auto', path=r'\\er-c-data-smb.fz-juelich.de\er-c-data\adhoc\livecalibration\20221025_154811\rotation_calibration.hdr')

In [5]:
overfocus_params = OverfocusParams(
    overfocus=0.001,  # m
    scan_pixel_size=0.000001,  # m
    camera_length=0.15,  # m
    detector_pixel_size=0.000050,  # m
    semiconv=0.020,  # rad
    #'scan_rotation': 37,
    scan_rotation=0.,
    flip_y=False,
    cy=ds.shape.sig[0] / 2,
    cx=ds.shape.sig[1] / 2,
)


In [6]:
keep_running = True
def on_click(btn):
    global keep_running
    keep_running = False
    btn.description = '👍 stopping'

stop_btn = ipywidgets.Button(description='Stop series')
stop_btn.on_click(on_click)

In [7]:
test_params = overfocus_params.copy()
test_params['scan_rotation'] = 330

process_params = OverfocusProcessParams(
    pair_distance=10.,
    pair_radius=1.,
)

In [8]:
overfocus_udf = OverfocusUDF(
    overfocus_params=test_params.copy(),
    process_params=process_params.copy(),
    point_y=127,
    point_x=127
)
sum_udf = SumUDF()

point_plot = BQLive2DPlot(dataset=ds, udf=overfocus_udf, channel='point')
shifted_sum_plot = BQLive2DPlot(dataset=ds, udf=overfocus_udf, channel='shifted_sum')
shifted_pair_plot = BQLive2DPlot(dataset=ds, udf=overfocus_udf, channel='shifted_pair')
sum_plot = BQLive2DPlot(dataset=ds, udf=overfocus_udf, channel='sum')
sum_plot_plain = BQLive2DPlot(dataset=ds, udf=sum_udf, channel='intensity')

plots = (point_plot, shifted_sum_plot, sum_plot_plain, shifted_pair_plot, sum_plot)

adjustment_outputs = []

for p in plots:
    # Capture the plots to display them in a grid later
    output = ipywidgets.Output()
    with output:
        p.display()
        # Some plot-specific tweaks for grid display
        if isinstance(p, BQLive2DPlot):
            p.figure.fig_margin={'top': 50, 'bottom': 0, 'left': 25, 'right': 25}
            p.figure.layout.width = '300px'
            p.figure.layout.height = '300px'
        elif isinstance(p, MPLLive2DPlot):
            p.fig.tight_layout()
            p.fig.set_size_inches((3, 3))
            p.fig.canvas.toolbar_position = 'bottom'
    adjustment_outputs.append(output)

rp = test_params
    
scan_rotation = ipywidgets.FloatSlider(min=0, max=360, description='Scan rotation / deg', value=rp['scan_rotation'])
flip_y = ipywidgets.Checkbox(description='Flip detector Y axis', value=rp['flip_y'])
overfocus = ipywidgets.FloatText(description='Overfocus / m', value=rp['overfocus'])
camera_length = ipywidgets.FloatText(description='Camera length / m', value=rp['camera_length'])
detector_pixel_size = ipywidgets.FloatText(description='Detector pixel size / m', value=rp['detector_pixel_size'])
scan_pixel_size = ipywidgets.FloatText(description='Scan pixel size / m', value=rp['scan_pixel_size'])

nav_y, nav_x = ds.shape.nav
pair_distance = ipywidgets.FloatSlider(min=0, max=np.sqrt(nav_y**2 + nav_x**2), description='Pair distance / px', value=process_params['pair_distance'])
pair_radius = ipywidgets.FloatText(description='Pair radius / px', value=process_params['pair_radius'])
    
ipywidgets.VBox([
    ipywidgets.HBox(adjustment_outputs[0:3]),
    ipywidgets.HBox(adjustment_outputs[3:5]),
    scan_rotation,
    flip_y,
    ipywidgets.HBox([overfocus, camera_length]),
    ipywidgets.HBox([detector_pixel_size, scan_pixel_size]),
    ipywidgets.HBox([pair_distance, pair_radius]),
    stop_btn
])

In [9]:
with ui_events() as poll:
    keep_running = True
    stop_btn.description = 'Stop series'
    while keep_running:
        tp = test_params
        tp['scan_rotation'] = scan_rotation.value
        tp['flip_y'] = flip_y.value
        tp['overfocus'] = overfocus.value
        tp['camera_length'] = camera_length.value
        tp['detector_pixel_size'] = detector_pixel_size.value
        tp['scan_pixel_size'] = scan_pixel_size.value
        pp = process_params
        pp['pair_distance'] = pair_distance.value
        pp['pair_radius'] = pair_radius.value
        result_iter = ctx.run_udf_iter(dataset=ds, udf=(overfocus_udf, sum_udf), plots=plots)
        for adjustment_res in result_iter:
            result_iter.update_parameters_experimental([
                {'overfocus_params': rp, 'process_params': pp},
                {}
            ])
            poll(100)
    stop_btn.description = 'Stopped'